In [1]:
####

In [2]:
import torch
from torch import nn

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class Conv(nn.Module):
    def __init__(self ,
                 in_channels , 
                 out_channels , 
                 kernel_size = (3 , 3) , 
                 stride = (1 , 1) , 
                 padding = 1 , 
                 use_norm = True , 
                 use_activation = True , 
                 use_pooling = False):
        super(Conv , self).__init__()

        self.use_norm = use_norm
        self.use_activation = use_activation
        self.use_pooling = use_pooling

        self.conv1 = nn.Conv2d(in_channels , 
                               out_channels ,
                               kernel_size , 
                               stride , 
                               padding)
        if self.use_norm:
            self.norm = nn.InstanceNorm2d(out_channels)
        if self.use_activation:
            self.activation = nn.LeakyReLU(0.2)

        if self.use_pooling:
            self.pooling = nn.MaxPool2d(kernel_size=(2 , 2) , stride=(2 , 2))

    def forward(self , x):

        x = self.conv1(x)
        if self.use_norm:
            x = self.norm(x)
        if self.use_activation:
            x = self.activation(x)
        if self.use_pooling:
            x = self.pooling(x)
        return x

In [5]:
class Darknet_Conv(nn.Module):
    def __init__(self ,
                 in_channels , 
                 out_channels):
        super(Darknet_Conv , self).__init__()

        self.conv1 = Conv(in_channels , 
                          in_channels , 
                          kernel_size = (1 , 1) , 
                          stride = (1 , 1) , 
                          padding = 0)
        self.conv2 = Conv(in_channels , in_channels)
        self.conv3 = Conv(in_channels , 
                          out_channels , 
                          kernel_size=(1 , 1) , 
                          stride = (1 ,1 ) , 
                          padding = 0)

    def forward(self , x):
        x_ = x.clone()
        x = self.conv1(x)
        x = self.conv2(x)
        x += x_
        x = self.conv3(x)
        return x

In [ ]:
x = torch.randn(2 , 3 , 64 , 64).to(device)
darknet_conv = Darknet_Conv(3 , 32).to(device)
z = darknet_conv(x)
z.shape

In [7]:
class DenseNet_Block(nn.Module):
    def __init__(self , 
                 in_channels , 
                 repeats , 
                 k = 12):
        super(DenseNet_Block , self).__init__()

        self.layers = nn.ModuleList()
        out_channels_last = 0
        in_channels_conv = in_channels // 3
        self.remaining_in_channels = in_channels - in_channels_conv

        self.in_channels_ = in_channels
        self.in_channels_conv_ = in_channels_conv
        for r in range(repeats):
            #in_channels = in_channels + out_channels_last
            out_channels = in_channels_conv + k 
            #print(in_channels , out_channels)
            self.layers.append(Darknet_Conv(in_channels_conv , 
                                             out_channels))
            
            out_channels_last += in_channels_conv
            in_channels_conv = out_channels + out_channels_last
        #print(out_channels)
        self.out_channels = out_channels + self.remaining_in_channels+ out_channels_last

        
    def _concat(self , x , prev):
        for prev_ in prev:
            x = torch.cat([x , prev_] , dim=1)
        return x

    def forward(self , x):
        x_cpy = x.clone()
        x = x[: , :self.in_channels_conv_ , : , :]
        x_ = x_cpy[: , self.in_channels_conv_ : self.in_channels_ , : , :]
        #print(x_.shape)
        prev = [x]
        for layer in self.layers:
            #print(x.shape)
            x = layer(x)
            x = self._concat(x , prev)
            prev.append(x)
        #print(x.shape , x_.shape)
        x = torch.cat([x , x_] , dim=1)
        return x

In [ ]:
densenet_block = DenseNet_Block(3 , 3).to(device)
x = torch.randn(2 , 3 , 112 , 112).to(device)
z = densenet_block(x)
z.shape

In [9]:
config = [
          [64 , (3 , 3) , (1 , 1) , 1 , True , True , True] ,  # [out_channels , kernel_size , stride , padding , norm , activation , pooling]
          1 , 
          [128 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          2 , 
          [256 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          8 , 
          [512 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          8 , 
          [1024 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          4
]       

In [10]:
class CSPDarknet(nn.Module):
    def __init__(self , 
                 in_channels , 
                 config = config):
        super(CSPDarknet , self).__init__()

        self.layers = nn.ModuleList()
        out_channels_list = [64 , 128 , 256 , 512 , 1024]
        i = 0
        for  layer in config:
            if isinstance(layer , list):
                out_channels , kernel_size , stride , padding , use_norm , use_activation , use_padding = layer
                self.layers.append(Conv(in_channels , out_channels , kernel_size , stride , padding , use_norm , use_activation , use_padding))
                in_channels = out_channels

            elif isinstance(layer , int):
                repeats = layer
                a = DenseNet_Block(in_channels , repeats)
                self.layers.append(a)
                in_channels = a.out_channels
                out_channels = out_channels_list[i]
                self.layers.append(Conv(in_channels , out_channels , kernel_size=(1 , 1) , stride=(1 , 1) , padding=0))
                i += 1
                in_channels = out_channels

                
    def forward(self , x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
cspdensenet = CSPDarknet(3).to(device)
x = torch.randn(2 , 3 , 416 , 416).to(device)
z = cspdensenet(x)
print(z.shape)